In [8]:
from random import SystemRandom
from sage.all import *
import numpy as np 
from pwn import * 
from tqdm import tqdm

host = 'socket.cryptohack.org' 
port = 13412

random = SystemRandom()

# dimension
n = 512
# plaintext modulus
p = 257
# ciphertext modulus
q = 6007
# message scaling factor
delta = int(round(q/p))

import json 
r = remote(host, port)

sigma = 3.8
normal = lambda: round(random.gauss(0, sigma))
uniform = lambda: random.randrange(q) - q//2
dtype = np.int64

def sample(shape, dist):
    return np.fromfunction(np.vectorize(lambda *_: dist()), shape).astype(dtype)

S = sample((n,), uniform)

rows = []
rhs = []

r.recvuntil(b'character in the flag?\n')

for i in tqdm(range(n)):
    payload = {
        "option": "encrypt",
        "message": "0"
    }
    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    A = eval(response["A"])
    b = int(response["b"])

    rows.append(A)
    rhs.append(b)

A = Matrix(ZZ, rows)
b = vector(ZZ, rhs)

M = (identity_matrix(512).augment(A.transpose())).stack(vector([0]*512 + list(-b)))  

M = M.LLL()
S = M[-1][:512]

print(S)
S = vector(ZZ, S)

flag = '' 
i = 0

while True:
    payload = {
        "option": "get_flag",
        "index": str(i)
    }

    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    A = eval(response["A"])
    A = vector(ZZ, A)

    b = int(response["b"])
    m = int(round((b - int((S * A))) / delta))

    flag += chr(m)
    print(flag)
    i += 1 



[x] Opening connection to socket.cryptohack.org on port 13412
[x] Opening connection to socket.cryptohack.org on port 13412: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13412: Done


100%|██████████| 512/512 [01:23<00:00,  6.13it/s]


(-1767, -1525, 1518, 1245, -710, 325, -423, -1272, 1023, 681, -2567, 2031, -758, -2713, 2675, -2152, -1454, 1416, -2990, -1245, 1459, -261, 1325, -2033, 1603, -2630, -2735, 1663, -913, -2201, -2992, 2280, -43, 2595, 2586, -6, -590, 459, -1494, -1408, -925, -2558, -914, -2596, -2399, 855, -2230, -43, 1319, 2266, -1268, 237, 953, -421, 1441, 871, 809, -975, -796, 2519, -2062, 2530, 295, -778, 2382, -321, -838, 616, 1674, 208, -1507, -2438, 2640, -962, -1420, -1990, -809, 729, -2541, -456, 2535, 33, 1748, 2952, 1709, 2936, 2923, -1088, -2486, 2494, 1414, 1391, 1858, 1971, 2826, -2408, 89, 902, -2914, -195, 855, 230, -1450, -223, -1746, -2560, -2718, -1283, 2556, -2873, -2221, 1536, 660, 2873, -2973, 2367, 1352, 1447, -1769, -2289, -1013, 2707, 574, -1961, -1678, 400, 2410, 596, -1710, 1100, -232, 1183, -1824, -1123, -2488, -1879, -2161, 1464, 1550, -849, 2776, 944, -1888, -1900, 839, -891, 1956, 998, 2214, 2534, -639, 2683, -1163, -294, 2986, -1219, -1946, -434, -1064, -942, 2275, -2802, 

KeyError: 'A'

In [ ]:
from random import SystemRandom
from sage.all import *
import numpy as np 
from pwn import * 
from tqdm import tqdm

host = 'socket.cryptohack.org' 
port = 13412

random = SystemRandom()

# dimension
n = 512
# plaintext modulus
p = 257
# ciphertext modulus
q = 6007
# message scaling factor
delta = int(round(q/p))

import json 
r = remote(host, port)

sigma = 3.8
normal = lambda: round(random.gauss(0, sigma))
uniform = lambda: random.randrange(q) - q//2
dtype = np.int64

def sample(shape, dist):
    return np.fromfunction(np.vectorize(lambda *_: dist()), shape).astype(dtype)

S = sample((n,), uniform)

rows = []
rhs = []

r.recvuntil(b'character in the flag?\n')

for i in tqdm(range(n)):
    payload = {
        "option": "encrypt",
        "message": "0"
    }
    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    A = eval(response["A"])
    b = int(response["b"])

    rows.append(A)
    rhs.append(b)

A = Matrix(ZZ, rows)
b = vector(ZZ, rhs)

M = Matrix(ZZ, n + 1)
for i in range(n):
    for j in range(n):
        M[j, i] = A[i, j]
    
for i in range(n):
    M[n, i] = b[i]

M = M.LLL()
print(M)

[x] Opening connection to socket.cryptohack.org on port 13412


[x] Opening connection to socket.cryptohack.org on port 13412: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13412: Done


100%|██████████| 512/512 [01:22<00:00,  6.18it/s]
